In [1]:



def _import(imports):
    result = ''
    for i in imports:
        result += combine(
        {
            'main': 'from {{ module }} import {{ component }}\n' if type(i) == list else 'import {{ module }}\n',
            'module': i[0] if type(i) == list else i,
            'component': i[1] if type(i) == list else ''
        })
    return result
def parameter_block(input):
    result = ''
    for i in input.keys():
        result += combine({
            'main': '{{ name }} = {{ value }},\n',
            'name': i,
            'value': input[i]
        })
    if(len(result) > 0):
        result = result[:-2]
    return result
from helper import combine
def model(name, description, x, y, layers, loss, batch_size, epochs, validation):
    def define(model, layers):
        imports = set()
        result = combine({
            'main': '{{ model }} = Sequential()\n',
            'model': model,
        })
        for layer in layers:
            layer_parameters = {
                'main': '{{ model }}.add({{ layer }})\n',
                'model': model,
            }
            if layer[0] == 'dense':
                layer_parameters['layer'] = 'Dense({{ size }}, activation=\'{{ activaction }}\')'
                layer_parameters['size'] = layer[1]
                layer_parameters['activaction'] = layer[2]
                imports.add('Dense')
            elif layer[0] == 'flatten':
                layer_parameters['layer'] = 'Flatten()'
                imports.add('Flatten')
            elif layer[0] == 'convolutional':
                layer_parameters['main'] = '{{ model }}.add({{ layer }})\n{{ model }}.add({{ layer }})\n'
                layer_parameters['model 2'] = model
                layer_parameters['layer'] = 'Conv2D({{ size }}, {{ kernel }}, activation=\'{{ activaction }}\')'
                layer_parameters['layer 2'] = 'MaxPooling2D({{ pool_size }}, {{ strides }})'
                layer_parameters['size'] = layer[1]
                layer_parameters['kernel'] = layer[2]
                layer_parameters['activaction'] = layer[3]
                layer_parameters['pool_size'] = layer[4]
                layer_parameters['strides'] = layer[5]
                imports.add('Conv2D')
                imports.add('MaxPooling2D')
            result += combine(layer_parameters)
        if len(result) > 0:
            result = result[:-1]
        return result, imports

    def compile(model, loss):
        return combine(
        { 
            'main': '''{{ model }}.compile(
    {{ parameter }}
)''',
            'parameter': parameter_block({
                'optimizer': '\'adam\'',
                'loss': '\'{{ loss }}\'',
                'metrics': '[\'accuracy\']'
            }),
            'model': model,
            'loss': loss
        })

    def fit(model, x, y, batch_size, epechs, validation, description = None):
        parameters = {
            'x': '{{ x }}',
            'y': '{{ y }}',
            'batch_size': '{{ batch_size }}',
            'epochs': '{{ epochs }}',
        }
        if validation != None:
            if type(validation) is tuple:
                parameters['validation_data'] = f'({validation[0]}, {validation[1]})'
            else:
                parameters['validation_split'] = validation
        if description:
            parameters['callbacks'] = '[TensorBoard(log_dir = \'logs/{{ model description }}\')]'
        return combine({
            'main': '''{{ model }}.fit(
    {{ parameter}}
)''',
            'parameter': parameter_block(parameters),
            'model': model,
            'x': x,
            'y': y,
            'batch_size': batch_size,
            'epochs': epechs,
            'model description': description})

    definition, imports = define(name, layers)

    result = _import([
        ['tensorflow.keras.models', 'Sequential'],
        ['tensorflow.keras.layers', ', '.join(imports)]
    ]) + '\n'
    result += definition + '\n\n'

    result += compile(name, loss) + '\n\n'

    result += fit(name, x, y, batch_size, epochs, validation, description) + '\n\n'

    result += combine({
        'main': '{{ model }}.save(\'{{ model description }}\')',
        'model': name,
        'model description': description
    
    })

    return result
['relu', 'softmax', 'sigmoid', 'tanh']
['mse', 'sparse_categorical_crossentropy', 'binary_crossentropy', 'categorical_crossentropy']

print(model('model','128x2model','x','y',[
        ['flatten'],
        ['dense','128','relu'],
        ['dense', '128', 'relu'],
        ['dense','10','softmax'],
        # ['convolutional','64','(3, 3)','relu', '(2, 2)', '(2, 2)']  
    ], 
    loss = 'sparse_categorical_crossentropy',
    batch_size = '30',
    epochs = '10',
    validation = '0.2'))
load = '{{ model }} = keras.models.load_model(\'{{ model description }}\')'

predict = '{{ model }}.predict({{ x }})'


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential()
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

model.fit(
    x = x,
    y = y,
    batch_size = 30,
    epochs = 10,
    validation_split = 0.2,
    callbacks = [TensorBoard(log_dir = 'logs/128x2model')]
)

model.save('128x2model')
